In [ ]:
---
layout: post
title: Final Blog for Sprint 4
description: Explanation of Sprint 4 all together
permalink: /final/blog/sprint4
menu: nav/tools_setup.html
toc: true
comments: true 
---

## 1. Introduction

This blog is about **Wellness Waypoints**, a program designed to help people find the nearest care centers (like hospitals or pharmacies) based on their injury and city. 

### Why This Program?
The program solves a simple problem: If you’re injured, where can you go for help? It finds the best places near you and shows them on a map. 

### What Did I Work On?
- I created the tools (backend) that store information about care centers.
- I built features (frontend) that let users search for care centers and see them on a map.
- I set up the database so it can be saved, backed up, and restored.

---

## 2. Input/Output Demonstrations

### What Happens on the Website?
Users can:
1. Select an injury from a dropdown.
2. Type in the name of their city.
3. Click "Go" to see a list of care centers and their locations on a map.

#### Code Example:
```javascript
async function FindLocations() {
  const location = document.getElementById("location").value.trim();
  const place = document.getElementById("place").value.trim();

  const url = `https://nominatim.openstreetmap.org/search?q=${location}+in+${place}&format=json&addressdetails=1&limit=10`;

  const response = await fetch(url);
  const data = await response.json();

  data.forEach((place, index) => {
    const row = `<tr>
        <td>${index + 1}</td>
        <td>${place.display_name.split(",")[0]}</td>
        <td>${place.display_name}</td>
        <td>${place.lat}, ${place.lon}</td>
      </tr>`;
    document.querySelector("#resultsTable tbody").innerHTML += row;
  });
}
```
- **What This Does:** This code takes the user’s input, asks a website (API) for care center locations, and shows the results in a table.

---

### Using Postman
Postman is a tool we use to test APIs. Here’s an example:
- **Request to Add a Care Center:**
  ```json
  {
    "injury": "Fracture",
    "location": "Hospital A",
    "address": "123 Main St"
  }
  ```
- **Response:**
  ```json
  {
    "id": 1,
    "injury": "Fracture",
    "location": "Hospital A",
    "address": "123 Main St"
  }
  ```

---

## 3. Backend Features

### Creating a Care Center
This code adds a new care center to the database:
```python
@token_required
def post(self):
    data = request.get_json()
    if not all(key in data for key in ["injury", "location", "address"]):
        return {"message": "Missing required fields"}, 400
    center = CareCenter(**data)
    center.create()
    return jsonify(center.read()), 201
```
- **What This Does:** It checks if all fields (injury, location, address) are provided and then saves the care center to the database.

### Database Queries
We use Python to ask the database for information. For example:
```python
centers = CareCenter.query.filter_by(location="Hospital A").all()
return jsonify([center.read() for center in centers])
```
- **What This Does:** It looks for care centers in "Hospital A" and sends the results back as a list.

---

## 4. Working with Code (Algorithms)

### Handling User Requests
This function sends user data (like an injury) to the server and saves it:
```javascript
async function postCareCenterData(injury, location, address) {
  const postData = { injury, location, address };

  try {
    const response = await fetch(`${pythonURI}/api/waypoints`, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(postData),
    });

    if (!response.ok) throw new Error("Failed to post data");

    const result = await response.json();
    console.log("Successfully posted:", result);
  } catch (error) {
    console.error("Error posting data:", error);
  }
}
```
- **What This Does:** Sends data to the backend and saves it. If there’s an error, it shows a message.

---

## 5. Challenges and Improvements

### Challenges
- Making the map update automatically.
- Handling errors when the API doesn’t work as expected.
- Debugging API responses to figure out what went wrong.

### Improvements
- Added better error messages for missing fields.
- Made the database queries faster.
- Improved how markers appear on the map.

---

## 6. Addressing Learning Requirements

- **Teamwork:** Built features that connect the frontend and backend.
- **Algorithms:** Used Python and JavaScript to manage user inputs, save data, and display results.
- **API Testing:** Used tools like Postman to check how the backend handles requests.
- **Database Management:** Created, backed up, and restored data using SQLAlchemy.

---

## 7. Third-Party Libraries

- **SQLAlchemy:** Makes working with databases easier.
- **Leaflet.js:** Displays care center locations on a map.
- **OpenStreetMap API:** Provides location data based on user input.

---

### Conclusion

The **Wellness Waypoints** project shows how coding can solve real-life problems. It connects users with care centers quickly and efficiently, making the program both useful and practical.
